## Predict Product Prices

### Week 7 Day 1

An introduction to LoRA and QLoRA



## A reminder of 2 important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Runtime menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [ ]:
# pip installs

#!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
#!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0
#!pip install -q datasets requests peft

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
# Constants

BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"
FINETUNED_MODEL = f"ed-donner/pricer-2024-09-13_13.04.39"

# Hyperparameters for QLoRA Fine-Tuning

LORA_R = 32
LORA_ALPHA = 64
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]

### Log in to HuggingFace

If you don't already have a HuggingFace account, visit https://huggingface.co to sign up and create a token.

Then select the Secrets for this Notebook by clicking on the key icon in the left, and add a new secret called `HF_TOKEN` with the value as your token.

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Trying out different Quantization



In [ ]:
# Load the Base Model without quantization

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto")

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## Restart your session!

In order to load the next model and clear out the cache of the last model, you'll now need to go to Runtime >> Restart session and run the initial cells (installs and imports and HuggingFace login) again.

This is to clean out the GPU.

In [ ]:
# Load the Base Model using 8 bit

quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

In [ ]:
base_model

## Restart your session!

In order to load the next model and clear out the cache of the last model, you'll now need to go to Runtime >> Restart session and run the initial cells (imports and HuggingFace login) again.

This is to clean out the GPU.

In [ ]:
# Load the Tokenizer and the Base Model using 4 bit

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
base_model

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)

In [ ]:
print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e9:,.2f} GB")

In [ ]:
fine_tuned_model

In [ ]:
# Each of the Target Modules has 2 LoRA Adaptor matrices, called lora_A and lora_B
# These are designed so that weights can be adapted by adding alpha * lora_A * lora_B
# Let's count the number of weights using their dimensions:

# See the matrix dimensions above
lora_q_proj = 4096 * 32 + 4096 * 32
lora_k_proj = 4096 * 32 + 1024 * 32
lora_v_proj = 4096 * 32 + 1024 * 32
lora_o_proj = 4096 * 32 + 4096 * 32

# Each layer comes to
lora_layer = lora_q_proj + lora_k_proj + lora_v_proj + lora_o_proj

# There are 32 layers
params = lora_layer * 32

# So the total size in MB is
size = (params * 4) / 1_000_000

print(f"Total number of params: {params:,} and size {size:,.1f}MB")

## 💾 Downloading Models to Run Locally

There are two main approaches to run these models on your local computer:

### Option 1: Download the Base Model (Llama 3.1 8B)

The base model is **meta-llama/Meta-Llama-3.1-8B** which is about **16GB** unquantized (or **4-5GB** with 4-bit quantization).

### Option 2: Download the Fine-tuned LoRA Adapter

The LoRA adapter is much smaller (**~50MB**) and works on top of the base model.

Both options are explained below!

### Method 1: Using Hugging Face CLI (Recommended)

This is the easiest way to download models from Hugging Face Hub to your local machine.

In [ ]:
# Step 1: Install huggingface-hub CLI (run this in your local terminal)
# pip install huggingface-hub

# Step 2: Login to Hugging Face (you'll need your HF token)
# huggingface-cli login

# Step 3: Download the base model (this will take a while - it's ~16GB)
# huggingface-cli download meta-llama/Meta-Llama-3.1-8B --local-dir ./models/llama-3.1-8b

# Step 4: Download the LoRA adapter (much smaller - ~50MB)
# huggingface-cli download ed-donner/pricer-2024-09-13_13.04.39 --local-dir ./models/pricer-lora

# The models will be saved to ./models/ directory
# You can then load them locally without internet!

### Method 2: Download Programmatically with Python

You can also download models using Python code, which gives you more control.

In [ ]:
from huggingface_hub import snapshot_download

# Download the base model
model_path = snapshot_download(
    repo_id="meta-llama/Meta-Llama-3.1-8B",
    local_dir="./models/llama-3.1-8b",
    token="your_hf_token_here"  # or use HF_TOKEN from environment
)

print(f"Base model downloaded to: {model_path}")

# Download the LoRA adapter
lora_path = snapshot_download(
    repo_id="ed-donner/pricer-2024-09-13_13.04.39",
    local_dir="./models/pricer-lora",
    token="your_hf_token_here"
)

print(f"LoRA adapter downloaded to: {lora_path}")

### Loading Models Locally After Download

Once you've downloaded the models, here's how to load them from your local directory:

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("./models/llama-3.1-8b")

# Load the base model with 4-bit quantization (to save memory)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

base_model = AutoModelForCausalLM.from_pretrained(
    "./models/llama-3.1-8b",  # Local path instead of repo name
    quantization_config=quant_config,
    device_map="auto",
)

# Load the LoRA adapter on top of the base model
model = PeftModel.from_pretrained(base_model, "./models/pricer-lora")

print("Models loaded from local storage!")
print(f"Memory footprint: {model.get_memory_footprint() / 1e9:.2f} GB")

### 💡 Pro Tips for Local Usage

**Storage Requirements:**
- Base model (unquantized): ~16GB
- Base model (4-bit quantized): ~4-5GB  
- LoRA adapter: ~50MB

**Memory Requirements (GPU/RAM):**
- With 4-bit quantization: ~6GB VRAM minimum
- With 8-bit quantization: ~10GB VRAM
- Without quantization: ~20GB VRAM

**Best Practices:**
1. Always use quantization (4-bit recommended) to save memory
2. Download LoRA adapter separately - it's tiny!
3. Keep models in a dedicated `./models/` directory
4. You can delete model files after downloading to save space
5. Use `device_map="auto"` to automatically distribute across GPU/CPU

**Running Without GPU:**
If you don't have a GPU, you can still run with CPU (it will be slower):
```python
base_model = AutoModelForCausalLM.from_pretrained(
    "./models/llama-3.1-8b",
    quantization_config=quant_config,
    device_map="cpu",  # Force CPU usage
)
```

**Alternative: Use Ollama Locally**
For easier local inference without Python, consider using Ollama:
```bash
# Install Ollama, then pull Llama 3.1
ollama pull llama3.1:8b
ollama run llama3.1:8b
```
This is simpler but doesn't support loading custom LoRA adapters directly.